In [1]:
import os
os.chdir('..')
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install Necessary Library and Data Loading

In [4]:
from sklearn.model_selection import train_test_split

# Load the CoNLL file
def load_conll_data(filepath):
    sentences = []
    sentence = []
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            if line.strip() == "":
                if sentence:
                    sentences.append(sentence)
                    sentence = []
            else:
                token, label = line.strip().split()
                sentence.append((token, label))
    if sentence:
        sentences.append(sentence)
    return sentences

# filepath = 'drive/MyDrive/Colab Notebooks/labeled_data.conll'
filepath = '/content/drive/MyDrive/Colab Notebooks/labeled_data.conll'
sentences = load_conll_data(filepath)

# Split into training and testing data
train_data, test_data = train_test_split(sentences, test_size=0.2, random_state=42)


In [5]:
print(sentences)

[[('እንኳን', 'O'), ('ለጌታችን', 'O'), ('ለመድሐኒታችን', 'O'), ('ለእየሱስ', 'O'), ('ክርስቶስ', 'O'), ('የጥምቀት', 'O'), ('ከተራ', 'O'), ('በዓል', 'O'), ('በሰላም', 'O'), ('አደረሰን', 'O'), ('አደረሳችሁ', 'O'), ('በዓሉ', 'O'), ('የሰለም', 'O'), ('የፍቅር', 'O'), ('የጤና', 'O'), ('ይሁንልን', 'O'), ('መልካም', 'O'), ('በዓል', 'O')], [('-', 'O'), ('-', 'O'), ('4-12', 'O'), ('-', 'O'), ('ዋጋ', 'B-PRICE'), ('850', 'I-PRICE'), ('ብር', 'B-PRICE'), ('ውስን', 'O'), ('ፍሬ', 'O'), ('ነው', 'O'), ('ያለው', 'O'), ('አድራሻ', 'O'), ('ቁ', 'B-LOC'), ('1', 'O'), ('መገናኛ', 'I-LOC'), ('መሰረት', 'I-LOC'), ('ደፋር', 'I-LOC'), ('ሞል', 'I-LOC'), ('ሁለተኛ', 'I-LOC'), ('ፎቅ', 'I-LOC'), ('ቢሮ', 'I-LOC'), ('ቁ', 'I-LOC'), ('05', 'I-LOC'), ('06', 'I-LOC'), ('ቁ', 'I-LOC'), ('2', 'O'), ('ፒያሳ', 'I-LOC'), ('ጊዮርጊስ', 'I-LOC'), ('አደባባይ', 'I-LOC'), ('ራመት_ታቦር_ኦዳ_ህንፃ', 'I-LOC'), ('1ኛ', 'O'), ('ፎቅ', 'I-LOC'), ('ሱቅ', 'I-LOC'), ('ቁ', 'I-LOC'), ('1', 'O'), ('-107', 'O'), ('0902660722', 'O'), ('0928460606', 'O'), ('ፒያሳ', 'I-LOC'), ('ቅርንጫፍ', 'I-LOC'), ('0941337070', 'O'), ('በ', 'O'), ('ለማዘዝ', 'O'), ('ይጠ

In [6]:
print(train_data)

[[('የህፃናት', 'O'), ('መመገቢያ', 'O'), ('ጡጦ', 'O'), ('ዋጋ', 'B-PRICE'), ('400', 'I-PRICE'), ('ብር', 'B-PRICE'), ('ውስን', 'O'), ('ፍሬ', 'O'), ('ነው', 'O'), ('ያለው', 'O'), ('አድራሻ', 'O'), ('ቁ', 'B-LOC'), ('1', 'O'), ('መገናኛ', 'I-LOC'), ('መሰረት', 'I-LOC'), ('ደፋር', 'I-LOC'), ('ሞል', 'I-LOC'), ('ሁለተኛ', 'I-LOC'), ('ፎቅ', 'I-LOC'), ('ቢሮ', 'I-LOC'), ('ቁ', 'I-LOC'), ('05', 'I-LOC'), ('06', 'I-LOC'), ('ቁ', 'I-LOC'), ('2', 'O'), ('ፒያሳ', 'I-LOC'), ('ጊዮርጊስ', 'I-LOC'), ('አደባባይ', 'I-LOC'), ('ራመት_ታቦር_ኦዳ_ህንፃ', 'I-LOC'), ('1ኛ', 'O'), ('ፎቅ', 'I-LOC'), ('ሱቅ', 'I-LOC'), ('ቁ', 'I-LOC'), ('1', 'O'), ('-107', 'O'), ('0902660722', 'O'), ('0928460606', 'O'), ('ፒያሳ', 'I-LOC'), ('ቅርንጫፍ', 'I-LOC'), ('0941337070', 'O'), ('በ', 'O'), ('ለማዘዝ', 'O'), ('ይጠቀሙ', 'O'), ('ለተጨማሪ', 'O'), ('ማብራሪያ', 'O'), ('የቴሌግራም', 'O'), ('ገፃችን', 'O')], [('ለፀጉሮ', 'O'), ('ወብት', 'O'), ('እና', 'O'), ('ለፀጉሮዎ', 'O'), ('ልስላሴ', 'O'), ('ጀኤስቲ', 'O'), ('የፀጉር', 'B-Product'), ('ማለስለሻ', 'O'), ('ለማዘዝ', 'O'), ('0974312223', 'O'), ('ይደውሉ', 'O'), ('ወይም', 'O'), ('_', 'O'), ('_'

In [7]:
model_names = {
    "xlm-roberta": "xlm-roberta-base",
    "distilbert": "distilbert-base-multilingual-cased",
    "mbert": "bert-base-multilingual-cased"
}

# Pre processing

In [25]:
from datasets import Dataset, load_dataset,DatasetDict

label_list = ['O', 'B-Product', 'I-Product', 'B-PRICE', 'I-PRICE', 'B-LOC', 'I-LOC']
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for i, label in enumerate(label_list)}

# Function to map labels to integers
def encode_labels(labels):
    return [label2id[label] for label in labels]
def convert_to_dataset(data):
    tokens = []
    labels = []

    for sentence in data:
        sentence_tokens = [pair[0] for pair in sentence]
        sentence_labels = encode_labels([pair[1] for pair in sentence])
        tokens.append(sentence_tokens)
        labels.append(sentence_labels)

    return Dataset.from_dict({"tokens": tokens, "labels": labels})

train_dataset = convert_to_dataset(train_data)
test_dataset = convert_to_dataset(test_data)

# # Create DatasetDict for train and test
datasets = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})



Tokenize the data and align the labels with tokens

In [26]:
from transformers import AutoTokenizer
from datasets import Dataset, DatasetDict

model = {}
tokenizer = {}
for key, model_name in model_names.items():
  model[key] = model_name
  tokenizer[key] = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [27]:
print(model)


{'xlm-roberta': 'xlm-roberta-base', 'distilbert': 'distilbert-base-multilingual-cased', 'mbert': 'bert-base-multilingual-cased'}


In [28]:
print(tokenizer['xlm-roberta'])

XLMRobertaTokenizerFast(name_or_path='xlm-roberta-base', vocab_size=250002, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	250001: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}
)


In [29]:
def tokenize_and_align_labels(datasets,tokenizer):
    tokenized_inputs = tokenizer(
        datasets["tokens"],
        truncation=True,
        padding="max_length",
        max_length=128,
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(datasets["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels = []
        previous_word_idx = None
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 for them.
            if word_idx is None:
                aligned_labels.append(-100)
            # Only label the first token of a given word.
            elif word_idx != previous_word_idx:
                aligned_labels.append(label[word_idx])
            # Set the label to -100 for the other subtokens of the word.
            else:
                aligned_labels.append(-100)
            previous_word_idx = word_idx

        labels.append(aligned_labels)

    tokenized_inputs["labels"] = labels
    # labelss=labels
    return tokenized_inputs
# Tokenize the datasets

tokenized_datasets = {}
for key, model_name in model_names.items():
  tokenized_datasets[key] = datasets.map(tokenize_and_align_labels, batched=True,fn_kwargs={'tokenizer': tokenizer[key]})



Map:   0%|          | 0/529 [00:00<?, ? examples/s]

Map:   0%|          | 0/133 [00:00<?, ? examples/s]

Map:   0%|          | 0/529 [00:00<?, ? examples/s]

Map:   0%|          | 0/133 [00:00<?, ? examples/s]

Map:   0%|          | 0/529 [00:00<?, ? examples/s]

Map:   0%|          | 0/133 [00:00<?, ? examples/s]

In [30]:
print(tokenized_datasets)

{'xlm-roberta': DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 529
    })
    test: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 133
    })
}), 'distilbert': DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 529
    })
    test: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 133
    })
}), 'mbert': DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 529
    })
    test: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 133
    })
})}


In [31]:
print(tokenized_datasets['xlm-roberta']['train'])

Dataset({
    features: ['tokens', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 529
})


In [32]:
print("Sample tokenized input with labels:", tokenized_datasets['xlm-roberta']['train'][0])
print("Length of tokens:", len(tokenized_datasets['xlm-roberta']['train'][0]["input_ids"]))
print("Length of labels:", len(tokenized_datasets['xlm-roberta']['train'][0]["labels"]))

Sample tokenized input with labels: {'tokens': ['የህፃናት', 'መመገቢያ', 'ጡጦ', 'ዋጋ', '400', 'ብር', 'ውስን', 'ፍሬ', 'ነው', 'ያለው', 'አድራሻ', 'ቁ', '1', 'መገናኛ', 'መሰረት', 'ደፋር', 'ሞል', 'ሁለተኛ', 'ፎቅ', 'ቢሮ', 'ቁ', '05', '06', 'ቁ', '2', 'ፒያሳ', 'ጊዮርጊስ', 'አደባባይ', 'ራመት_ታቦር_ኦዳ_ህንፃ', '1ኛ', 'ፎቅ', 'ሱቅ', 'ቁ', '1', '-107', '0902660722', '0928460606', 'ፒያሳ', 'ቅርንጫፍ', '0941337070', 'በ', 'ለማዘዝ', 'ይጠቀሙ', 'ለተጨማሪ', 'ማብራሪያ', 'የቴሌግራም', 'ገፃችን'], 'labels': [-100, 0, -100, -100, 0, -100, -100, -100, 0, -100, -100, 3, 4, 3, 0, -100, -100, 0, -100, 0, 0, 0, 5, 0, 6, -100, -100, 6, 6, -100, 6, -100, 6, -100, 6, -100, -100, 6, 6, 6, 6, 6, 0, 6, -100, -100, -100, 6, -100, 6, 6, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 0, -100, 6, -100, -100, 6, -100, -100, 6, 0, 0, -100, 0, -100, -100, -100, 0, -100, -100, -100, 6, -100, -100, -100, 6, -100, -100, -100, 0, -100, -100, -100, 0, 0, -100, -100, 0, -100, 0, -100, -100, 0, 0, -100, -100, -100, -100, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100

In [33]:
print("Sample tokenized input with labels:", tokenized_datasets['xlm-roberta']['train'][0])
print("Length of tokens:", len(tokenized_datasets['xlm-roberta']['train'][0]["input_ids"]))
print("Length of labels:", len(tokenized_datasets['xlm-roberta']['train'][0]["labels"]))

Sample tokenized input with labels: {'tokens': ['የህፃናት', 'መመገቢያ', 'ጡጦ', 'ዋጋ', '400', 'ብር', 'ውስን', 'ፍሬ', 'ነው', 'ያለው', 'አድራሻ', 'ቁ', '1', 'መገናኛ', 'መሰረት', 'ደፋር', 'ሞል', 'ሁለተኛ', 'ፎቅ', 'ቢሮ', 'ቁ', '05', '06', 'ቁ', '2', 'ፒያሳ', 'ጊዮርጊስ', 'አደባባይ', 'ራመት_ታቦር_ኦዳ_ህንፃ', '1ኛ', 'ፎቅ', 'ሱቅ', 'ቁ', '1', '-107', '0902660722', '0928460606', 'ፒያሳ', 'ቅርንጫፍ', '0941337070', 'በ', 'ለማዘዝ', 'ይጠቀሙ', 'ለተጨማሪ', 'ማብራሪያ', 'የቴሌግራም', 'ገፃችን'], 'labels': [-100, 0, -100, -100, 0, -100, -100, -100, 0, -100, -100, 3, 4, 3, 0, -100, -100, 0, -100, 0, 0, 0, 5, 0, 6, -100, -100, 6, 6, -100, 6, -100, 6, -100, 6, -100, -100, 6, 6, 6, 6, 6, 0, 6, -100, -100, -100, 6, -100, 6, 6, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 0, -100, 6, -100, -100, 6, -100, -100, 6, 0, 0, -100, 0, -100, -100, -100, 0, -100, -100, -100, 6, -100, -100, -100, 6, -100, -100, -100, 0, -100, -100, -100, 0, 0, -100, -100, 0, -100, 0, -100, -100, 0, 0, -100, -100, -100, -100, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100

In [34]:
from transformers import XLMRobertaForTokenClassification, DistilBertForTokenClassification, BertForTokenClassification
from transformers import Trainer, TrainingArguments
models = {}
models['xlm-roberta'] = XLMRobertaForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=len(label_list))
models["distilbert"] = DistilBertForTokenClassification.from_pretrained("distilbert-base-multilingual-cased", num_labels=len(label_list))
models["mbert"] = BertForTokenClassification.from_pretrained("bert-base-multilingual-cased", num_labels=len(label_list))


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
print(models['distilbert'])

DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
  

In [38]:
from transformers import Trainer, TrainingArguments
from seqeval.metrics import classification_report


def fine_tune_model(model, model_name):
  training_args = TrainingArguments(
      output_dir=f"./results/{model_name}",
      evaluation_strategy="epoch",
      logging_strategy="steps",
      logging_steps=3,
      learning_rate=2e-5,
      per_device_train_batch_size=8,
      per_device_eval_batch_size=8,
      num_train_epochs=20,
      weight_decay=0.01,
      logging_dir='./logs',
  )
  # Define a Trainer
  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=tokenized_datasets[model_name]['train'],
      eval_dataset=tokenized_datasets[model_name]['test'],
      tokenizer=tokenizer[model_name],
  )

  # Train the model
  trainer.train()
  return trainer


In [39]:
roberta_trainer = fine_tune_model(models['xlm-roberta'], 'xlm-roberta')

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: ERROR API key must be 40 characters long, yours was 15


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,0.131200,0.124958
2,0.049500,0.057806
3,0.013200,0.052805
4,0.029400,0.028695
5,0.069500,0.022253
6,0.013800,0.026583
7,0.027100,0.025156
8,0.016800,0.011450
9,0.002000,0.012335
10,0.005000,0.014817


# Model Training

In [40]:
distilbert_trainer = fine_tune_model(models['distilbert'], 'distilbert')

Epoch,Training Loss,Validation Loss
1,0.377600,0.336800
2,0.197500,0.159567
3,0.099100,0.120251
4,0.116500,0.101191
5,0.180300,0.091101
6,0.070700,0.086130
7,0.102800,0.078193
8,0.124400,0.087715
9,0.027200,0.074295
10,0.070600,0.076956


In [41]:
mbert_trainer = fine_tune_model(models['mbert'], 'mbert')

Epoch,Training Loss,Validation Loss
1,0.350400,0.283484
2,0.201800,0.144081
3,0.080700,0.107935
4,0.103800,0.084526
5,0.163100,0.083552
6,0.080300,0.097453
7,0.102100,0.074235
8,0.087200,0.069243
9,0.040100,0.057259
10,0.076900,0.055799


# Evaluation

In [42]:
import numpy as np
from seqeval.metrics import classification_report

pred_data = {'xlm-roberta':roberta_trainer,'distilbert':distilbert_trainer,'mbert':mbert_trainer}
# Make predictions on the test set
for model_name,trainer in pred_data.items():
  predictions, labels, _ = trainer.predict(tokenized_datasets[model_name]["test"])
  predictions = np.argmax(predictions, axis=2)

  # Convert predictions and labels to original label format
  true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
  predicted_labels = [[label_list[p] for (p, l) in zip(pred, label) if l != -100] for pred, label in zip(predictions, labels)]

  # Evaluate using seqeval
  report = classification_report(true_labels, predicted_labels)
  print(f'{model_name} evaluation \n{report}\n\n')


xlm-roberta evaluation 
              precision    recall  f1-score   support

         LOC       1.00      1.00      1.00       276
       PRICE       0.97      1.00      0.99        76
     Product       0.97      0.95      0.96        41

   micro avg       0.99      0.99      0.99       393
   macro avg       0.98      0.98      0.98       393
weighted avg       0.99      0.99      0.99       393





distilbert evaluation 
              precision    recall  f1-score   support

         LOC       1.00      0.99      0.99       303
       PRICE       0.84      0.92      0.88        76
     Product       0.62      0.42      0.50        50

   micro avg       0.94      0.91      0.92       429
   macro avg       0.82      0.78      0.79       429
weighted avg       0.93      0.91      0.92       429





mbert evaluation 
              precision    recall  f1-score   support

         LOC       1.00      0.99      1.00       303
       PRICE       0.84      0.93      0.88        76
     Product       0.59      0.52      0.55        50

   micro avg       0.92      0.93      0.93       429
   macro avg       0.81      0.82      0.81       429
weighted avg       0.92      0.93      0.92       429





# Save The Model

In [43]:
best_model_name = 'xlm-roberta'

In [44]:
models[best_model_name].save_pretrained("drive/MyDrive/Colab Notebooks/finetuned-model")
tokenizer[best_model_name].save_pretrained("drive/MyDrive/Colab Notebooks/finetuned-model")


('drive/MyDrive/Colab Notebooks/finetuned-model/tokenizer_config.json',
 'drive/MyDrive/Colab Notebooks/finetuned-model/special_tokens_map.json',
 'drive/MyDrive/Colab Notebooks/finetuned-model/sentencepiece.bpe.model',
 'drive/MyDrive/Colab Notebooks/finetuned-model/added_tokens.json',
 'drive/MyDrive/Colab Notebooks/finetuned-model/tokenizer.json')